# My project looks at complaints for rent stabilized apartments in Manhattan and Broojlyn Complains.
from Jan 01 12am 2020 to april 11 5pm.


I'm guessing that complaints increased eaach year since 2021 as Landlords try to make more money from their properties. Landlords claim they need to do improvements so that they can up the cost based on the remodel. They may also like a new tenant so they can adjust their property price to the current market value.

PDFs --
Manhattan: https://rentguidelinesboard.cityofnewyork.us/wp-content/uploads/2024/11/2023-DHCR-Bldg-File-Manhattan.pdf
Brooklyn: https://rentguidelinesboard.cityofnewyork.us/wp-content/uploads/2024/11/2023-DHCR-Bldg-File-Brooklyn.pdf

In [199]:
import pandas as pd

In [201]:
## call in my complaints data

In [203]:
df_complaintsraw = pd.read_csv('Housing_Maintenance_Code_Complaints_and_Problems_20250411.csv')

In [204]:
df_complaintsraw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1869009 entries, 0 to 1869008
Data columns (total 33 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   Received Date             object 
 1   Problem ID                int64  
 2   Complaint ID              int64  
 3   Building ID               int64  
 4   Borough                   object 
 5   House Number              object 
 6   Street Name               object 
 7   Post Code                 float64
 8   Block                     int64  
 9   Lot                       int64  
 10  Apartment                 object 
 11  Community Board           int64  
 12  Unit Type                 object 
 13  Space Type                object 
 14  Type                      object 
 15  Major Category            object 
 16  Minor Category            object 
 17  Problem Code              object 
 18  Complaint Status          object 
 19  Complaint Status Date     object 
 20  Problem Status          

In [207]:
len(df_complaintsraw)

1869009

In [209]:
df_complaintsraw[['Received Date','Block', 'Lot','Space Type','Type','Major Category','Minor Category', 'Problem Code','Problem Status','Problem Duplicate Flag','NTA']]

,Received Date,Block,Lot,Space Type,Type,Major Category,Minor Category,Problem Code,Problem Status,Problem Duplicate Flag,NTA
0,01/01/2020,2215,326,KITCHEN,NON EMERGENCY,PLUMBING,WATER SUPPLY,NO HOT WATER,CLOSE,N,Kingsbridge-Marble Hill
1,01/01/2020,2095,23,ENTIRE APARTMENT,EMERGENCY,HEAT/HOT WATER,APARTMENT ONLY,NO HEAT,CLOSE,Y,Hamilton Heights-Sugar Hill
2,01/01/2020,862,11,BUILDING-WIDE,EMERGENCY,HEAT/HOT WATER,ENTIRE BUILDING,NO HEAT,CLOSE,N,Midtown South-Flatiron-Union Square
3,01/01/2020,2215,326,ENTIRE APARTMENT,EMERGENCY,DOOR/WINDOW,DOOR,BROKEN OR MISSING,CLOSE,N,Kingsbridge-Marble Hill
4,01/01/2020,244,40,ENTRANCE/FOYER,EMERGENCY,DOOR/WINDOW,WINDOW PANE,GLASS BROKEN OR MISSING,CLOSE,N,Lower East Side
...,...,...,...,...,...,...,...,...,...,...,...
1869004,04/10/2025,1879,43,BUILDING-WIDE,EMERGENCY,HEAT/HOT WATER,ENTIRE BUILDING,NO HOT WATER,OPEN,Y,Upper West Side-Manhattan Valley
1869005,04/10/2025,1999,15,ENTIRE APARTMENT,EMERGENCY,HEAT/HOT WATER,APARTMENT ONLY,NO HEAT,OPEN,N,Bedford-Stuyvesant (West)
1869006,04/10/2025,923,39,ENTIRE APARTMENT,EMERGENCY,HEAT/HOT WATER,APARTMENT ONLY,NO HEAT,OPEN,Y,Gramercy
1869007,04/11/2025,2054,79,BUILDING-WIDE,EMERGENCY,HEAT/HOT WATER,ENTIRE BUILDING,NO HOT WATER,OPEN,N,Hamilton Heights-Sugar Hill


In [211]:
## how many major category types are there?
df_complaintsraw["Major Category"].nunique()

14

In [213]:
df_complaintsraw["Major Category"].unique()

array(['PLUMBING', 'HEAT/HOT WATER', 'DOOR/WINDOW', 'WATER LEAK',
       'GENERAL', 'FLOORING/STAIRS', 'ELECTRIC', 'PAINT/PLASTER',
       'UNSANITARY CONDITION', 'APPLIANCE', 'SAFETY', 'ELEVATOR',
       'OUTSIDE BUILDING', 'LINE OF TRAVEL'], dtype=object)

In [215]:
## how many minor category types are there?
df_complaintsraw["Minor Category"].nunique()

## okay so a more detailed look at the general main/"major" category and problem code is even more decriptive

59

In [217]:
## how many space types??
df_complaintsraw["Space Type"].nunique()

33

In [219]:
## save it as the DataFrame i'll be working in

complaints_df = df_complaintsraw[['Received Date','Block', 'Lot','Space Type','Type','Major Category','Minor Category', 'Problem Code','Problem Status','Problem Duplicate Flag','NTA']]

In [221]:
# import my list of blocks and lots to vlookup

df_stabilizedraw = pd.read_excel('Rent Stabilized Buildings in NYC.xlsx', sheet_name='Block and Lot')

In [222]:

df_stabilizedraw

,Block,Lot,Rent Stabilized
0,2290,17.0,Yes
1,2290,19.0,Yes
2,2290,20.0,Yes
3,2290,22.0,Yes
4,2290,23.0,Yes
...,...,...,...
30027,759,7.0,Yes
30028,733,23.0,Yes
30029,733,8.0,Yes
30030,732,68.0,Yes


In [223]:
#only showing rent stabilized apartments by merging dfs
## renaming simply because it has all of the information that i want/need
df = complaints_df.merge(df_stabilizedraw, on=['Block','Lot'], how='inner')

In [224]:
## puts datetime format for the received date column??
df['Received Date'] = pd.to_datetime(df['Received Date'], format='%m/%d/%Y')

In [229]:
## save it
df['Year'] = df['Received Date'].dt.year

In [231]:
#break down of numbers of complaints by year

df.groupby(df['Received Date'].dt.year)['Rent Stabilized'].count()

Received Date
2020    223416
2021    296053
2022    357106
2023    345471
2024    393266
2025    138076
Name: Rent Stabilized, dtype: int64

In [233]:
## make it look nicer and add commas
df.groupby(df['Year'])['Rent Stabilized'].count().to_frame("Number of Complaints by year")



,Number of Complaints by year
Year,
2020,223416
2021,296053
2022,357106
2023,345471
2024,393266
2025,138076


## how do i add commas???

In [235]:
## largest to smallest
df.groupby(df['Year'])['Rent Stabilized'].count().nlargest().to_frame("Number of Complaints by year")


,Number of Complaints by year
Year,
2024,393266
2022,357106
2023,345471
2021,296053
2020,223416


In [237]:
###What was the most common complaint type???

df.groupby(['Year', 'Type']).size().to_frame("Number of Emergencies By Year")

Number of Emergencies By Year
Year Type                                              
2020 EMERGENCY                                   142252
     IMMEDIATE EMERGENCY                           8074
     NON EMERGENCY                                73090
2021 EMERGENCY                                   182794
     IMMEDIATE EMERGENCY                          10798
     NON EMERGENCY                               102461
2022 EMERGENCY                                   230082
     IMMEDIATE EMERGENCY                          11866
     NON EMERGENCY                               115158
2023 EMERGENCY                                   207021
     IMMEDIATE EMERGENCY                          11107
     NON EMERGENCY                               127343
2024 EMERGENCY                                   237419
     IMMEDIATE EMERGENCY                          13417
     NON EMERGENCY                               142430
2025 EMERGENCY                                    97618
     IMMEDIATE EMERGENCY                           4209
     NON EMERGENCY                                36249

In [87]:
## which year has the most regular emergency
df.groupby(['Year']) ['Type'].size().nlargest().to_frame("Number of Emergencies By Year")

,Number of Emergencies By Year
Year,
2024,393266
2022,357106
2023,345471
2021,296053
2020,223416


In [239]:
len(df)

1753388

In [241]:
## which major category is the most common
df.groupby("Major Category").size().nlargest().to_frame("MAJOR CATEGORY NUMBERS BY YEAR")

## most common is hot water

,MAJOR CATEGORY NUMBERS BY YEAR
Major Category,
HEAT/HOT WATER,645449
UNSANITARY CONDITION,269106
PLUMBING,158037
PAINT/PLASTER,156856
DOOR/WINDOW,115222


In [243]:
## what is the most  common complaint per year
df.groupby(['Year', 'Major Category']).size().nlargest().to_frame("Number of Emergencies By Year")

,,Number of Emergencies By Year
Year,Major Category,
2022,HEAT/HOT WATER,140554
2024,HEAT/HOT WATER,130444
2023,HEAT/HOT WATER,113205
2021,HEAT/HOT WATER,104660
2020,HEAT/HOT WATER,87242


In [245]:
df['Major Category'].unique()


array(['HEAT/HOT WATER', 'DOOR/WINDOW', 'WATER LEAK', 'GENERAL',
       'FLOORING/STAIRS', 'ELECTRIC', 'PAINT/PLASTER', 'PLUMBING',
       'UNSANITARY CONDITION', 'APPLIANCE', 'SAFETY', 'ELEVATOR',
       'OUTSIDE BUILDING', 'LINE OF TRAVEL'], dtype=object)

In [247]:
## how many complaints were filed Jan to April of previous years too

## create a database that only has certain months (Jan 01 12am 2020 to april 11 5pm)
## define dates 

start_2020 = '01-01-2020'
end_2020 = '04-11-2020'

start_2021 = '01-01-2021'
end_2021 = '04-11-2021'

start_2022 = '01-01-2022'
end_2022 = '04-11-2022'

start_2023 = '01-01-2023'
end_2023 = '04-11-2023'

start_2024 = '01-01-2024'
end_2024 = '04-11-2024'

start_2025 = '01-01-2025'
end_2025 = '04-11-2025'



In [253]:
earlydate_df = df[
(df['Received Date'] >= start_2020) & (df['Received Date'] <= end_2020) |
(df['Received Date'] >= start_2021) & (df['Received Date'] <= end_2021) |
(df['Received Date'] >= start_2022) & (df['Received Date'] <= end_2022) |
(df['Received Date'] >= start_2023) & (df['Received Date'] <= end_2023) |
(df['Received Date'] >= start_2024) & (df['Received Date'] <= end_2024) |
(df['Received Date'] >= start_2025) & (df['Received Date'] <= end_2025)
] 

## call it
earlydate_df


,Received Date,Block,Lot,Space Type,Type,Major Category,Minor Category,Problem Code,Problem Status,Problem Duplicate Flag,NTA,Rent Stabilized,Year
0,2020-01-01,862,11,BUILDING-WIDE,EMERGENCY,HEAT/HOT WATER,ENTIRE BUILDING,NO HEAT,CLOSE,N,Midtown South-Flatiron-Union Square,Yes,2020
1,2020-01-01,244,40,ENTRANCE/FOYER,EMERGENCY,DOOR/WINDOW,WINDOW PANE,GLASS BROKEN OR MISSING,CLOSE,N,Lower East Side,Yes,2020
2,2020-01-01,1960,22,BUILDING-WIDE,EMERGENCY,HEAT/HOT WATER,ENTIRE BUILDING,NO HEAT,CLOSE,Y,Clinton Hill,Yes,2020
3,2020-01-01,7930,1,ENTIRE APARTMENT,EMERGENCY,HEAT/HOT WATER,APARTMENT ONLY,NO HEAT,CLOSE,N,Flatlands,Yes,2020
4,2020-01-01,7930,1,ENTIRE APARTMENT,EMERGENCY,HEAT/HOT WATER,APARTMENT ONLY,NO HEAT,CLOSE,N,Flatlands,Yes,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1753383,2025-04-10,1879,43,BUILDING-WIDE,EMERGENCY,HEAT/HOT WATER,ENTIRE BUILDING,NO HOT WATER,OPEN,Y,Upper West Side-Manhattan Valley,Yes,2025
1753384,2025-04-10,1999,15,ENTIRE APARTMENT,EMERGENCY,HEAT/HOT WATER,APARTMENT ONLY,NO HEAT,OPEN,N,Bedford-Stuyvesant (West),Yes,2025
1753385,2025-04-10,923,39,ENTIRE APARTMENT,EMERGENCY,HEAT/HOT WATER,APARTMENT ONLY,NO HEAT,OPEN,Y,Gramercy,Yes,2025
1753386,2025-04-11,2054,79,BUILDING-WIDE,EMERGENCY,HEAT/HOT WATER,ENTIRE BUILDING,NO HOT WATER,OPEN,N,Hamilton Heights-Sugar Hill,Yes,2025


In [261]:
## make comparisons
earlytotals_df = earlydate_df.groupby(['Year']).size().nlargest().to_frame("Number of Emergencies By Apr 11")


In [263]:
##call it

earlytotals_df

,Number of Emergencies By Apr 11
Year,
2025,138076
2024,131067
2022,129065
2023,103377
2021,91514


In [269]:
## finding the average before 2025, lets get rid of 2025 data
earlydate_df = df[
(df['Received Date'] >= start_2020) & (df['Received Date'] <= end_2020) |
(df['Received Date'] >= start_2021) & (df['Received Date'] <= end_2021) |
(df['Received Date'] >= start_2022) & (df['Received Date'] <= end_2022) |
(df['Received Date'] >= start_2023) & (df['Received Date'] <= end_2023) |
(df['Received Date'] >= start_2024) & (df['Received Date'] <= end_2024) 
]

## call it
earlydate_df



,Received Date,Block,Lot,Space Type,Type,Major Category,Minor Category,Problem Code,Problem Status,Problem Duplicate Flag,NTA,Rent Stabilized,Year
0,2020-01-01,862,11,BUILDING-WIDE,EMERGENCY,HEAT/HOT WATER,ENTIRE BUILDING,NO HEAT,CLOSE,N,Midtown South-Flatiron-Union Square,Yes,2020
1,2020-01-01,244,40,ENTRANCE/FOYER,EMERGENCY,DOOR/WINDOW,WINDOW PANE,GLASS BROKEN OR MISSING,CLOSE,N,Lower East Side,Yes,2020
2,2020-01-01,1960,22,BUILDING-WIDE,EMERGENCY,HEAT/HOT WATER,ENTIRE BUILDING,NO HEAT,CLOSE,Y,Clinton Hill,Yes,2020
3,2020-01-01,7930,1,ENTIRE APARTMENT,EMERGENCY,HEAT/HOT WATER,APARTMENT ONLY,NO HEAT,CLOSE,N,Flatlands,Yes,2020
4,2020-01-01,7930,1,ENTIRE APARTMENT,EMERGENCY,HEAT/HOT WATER,APARTMENT ONLY,NO HEAT,CLOSE,N,Flatlands,Yes,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353108,2024-04-11,2164,38,ENTIRE APARTMENT,EMERGENCY,HEAT/HOT WATER,APARTMENT ONLY,NO HOT WATER,CLOSE,N,Washington Heights (North),Yes,2024
1353109,2024-04-11,2054,79,BUILDING-WIDE,EMERGENCY,HEAT/HOT WATER,ENTIRE BUILDING,NO HOT WATER,CLOSE,Y,Hamilton Heights-Sugar Hill,Yes,2024
1353110,2024-04-11,395,59,KITCHEN,NON EMERGENCY,WATER LEAK,SLOW LEAK,AT WALL OR CEILING,CLOSE,N,Carroll Gardens-Cobble Hill-Gowanus-Red Hook,Yes,2024
1353111,2024-04-11,395,59,KITCHEN,EMERGENCY,PLUMBING,BASIN/SINK,PIPE LEAKING,CLOSE,N,Carroll Gardens-Cobble Hill-Gowanus-Red Hook,Yes,2024


In [273]:
## call it
earlytotals_df = earlydate_df.groupby(['Year']).size().nlargest().to_frame("Number of Emergencies By Apr 11")
earlytotals_df


,Number of Emergencies By Apr 11
Year,
2024,131067
2022,129065
2023,103377
2021,91514
2020,77763


In [275]:
earlytotals_df.mean()

Number of Emergencies By Apr 11    106557.2
dtype: float64

In [303]:
## find the percent change between 138076 (2025) and 106557.2 (previous years' average)

percent_change = (138076 - 106557.2) / 106557.2 *100

percent_change

29.579230685490987

In [305]:
print(f"2025 has had 138076 compaints from Jan 1st to Apr 11. This is a {percent_change}% change in comparison to the average amount of complaints by this time from 2020-2024.")

2025 has had 138076 compaints from Jan 1st to Apr 11. This is a 29.579230685490987% change in comparison to the average amount of complaints by this time from 2020-2024.
